# 公式Tutorialの実行

## 手順の概要

Watson NLC 公式ドキュメントからの引用です。

https://www.ibm.com/watson/developercloud/doc/natural-language-classifier/getting-started.html

<img src="images/oper0217_03.png" />

### (1) 認証情報取得

- Bluemixにログインし、NLCサービスを作成
- サービス作成時に発行される認証情報（Credentials）を使用します

上記の手順は<b><a href="Watson_NLC_00.ipynb">こちら</a></b>になります。

### (2) 分類器作成

- サンプルの訓練データ（CSV）をダウンロード
- 日本語に変えてみます <---公式ページにない手順
- 分類器作成を開始させるAPI（POST /classifiers/）を実行

分類器の作成開始から完了までは時間を要します。

分類器作成が完了したかどうかは、完了したかどうかを照会するAPIで、逐一問い合わせる必要があります。

### (3) 分類実行

- 訓練が完了した分類器に対し、分類したいテキストを送信
- 該当する上位クラスと、マッチ率（confidence）がJSON形式で戻ります

該当する上位１０件までがリストされるようです。

またTutorialの例では、訓練データに入っていない単語が含まれているテキストでも、分類が可能な旨の記述がありますが・・・

### (4) 分類器の削除

- 上記までの試行が完了したら、分類器を削除

忘れないうちに削除しておきましょう・・・ということなのかと思われます。

## 依存ライブラリー導入

Python3からNLCのAPIをコールするためには、依存ライブラリー「watson_developer_cloud」を追加導入する必要がございます。

コマンドラインから「pip3 install watson_developer_cloud」と実行し、導入できるかと存じます。

~~~~
MacBookPro-makmorit-jp:~ makmorit$ pip3 install watson_developer_cloud
Collecting watson-developer-cloud
  Downloading watson-developer-cloud-0.23.0.tar.gz (52kB)
    100% |████████████████████████████████| 61kB 451kB/s 
Requirement already satisfied (use --upgrade to upgrade): requests<3.0,>=2.0 in /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages (from watson-developer-cloud)
Collecting pysolr<4.0,>=3.3 (from watson-developer-cloud)
  Downloading pysolr-3.6.0-py2.py3-none-any.whl
Building wheels for collected packages: watson-developer-cloud
  Running setup.py bdist_wheel for watson-developer-cloud ... done
  Stored in directory: /Users/makmorit/Library/Caches/pip/wheels/e2/e9/4e/bdd7ab2ed130d3ed16ac583d4843de1335f7f03f531d6ba01d
Successfully built watson-developer-cloud
Installing collected packages: pysolr, watson-developer-cloud
Successfully installed pysolr-3.6.0 watson-developer-cloud-0.23.0
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
MacBookPro-makmorit-jp:~ makmorit$
~~~~

## 実行例

上記のライブラリー「watson_developer_cloud」を使用し、Python3でTutorialを実行してみます。

もともとWeb APIの戻り値がJSON形式であるため、Python3との親和性は高いかと存じます。

### クラス「NaturalLanguageClassifierV1」生成

「NaturalLanguageClassifierV1」クラスのコンストラクターを実行すると、サービス・インスタンス（＝インスタンス課金の単位）の参照（下記例では変数 natural_language_classifier）が戻されます。

以降のNLC APIコールは、このサービス・インスタンス参照を使用して実行することになります。

なお、コンストラクターの引数には、サービス作成時に発行された認証情報（username／password）を設定します。

In [2]:
import json
from watson_developer_cloud import NaturalLanguageClassifierV1

natural_language_classifier = NaturalLanguageClassifierV1(
  username='7fdb13f0-1c16-48f5-9a7d-6374ee56925d',
  password='LLwyqghs3cPB')

すでに作成されている分類器を確認するには、list関数を使用します。

（いまはまだ作成されていないので、空リストが戻ります）

In [2]:
classifiers = natural_language_classifier.list()
classifiers

{'classifiers': []}

### 訓練データの作成

例を簡単にするのと「日本語もサポート」と謳っている仕様の検証のため、ダウンロードしたサンプルを日本語に書き換えます。

#### ダウンロードしたサンプルデータ

~~~~
How hot is it today?,temperature
Is it hot outside?,temperature
Will it be uncomfortably hot?,temperature
：
What is today's expected humidity?,conditions
Will the blizzard hit us?,conditions
Is it drizzling?,conditions
~~~~



#### 日本語化したサンプルデータ

形態素解析が必要となります（御社テストデータを、MeCab にかけた例。仕込み用コードは<b><a href="Watson_NLC_99.ipynb">こちら</a></b>）

~~~~
：
"試用 期間 は ある ？ ",2657
"試用 期間 は ？ ",2657
"試用 期間 は 有り ます か ？ ",2657
"初期 費用 月額 費用 以外 に 掛かる 費用 は ？ ",2729
"初期 費用 、 月額 費用 以外 は ？ ",2729
"初期 費用 、 月額 費用 以外 に かかり ます か ？ ",2729
"導入 を 考え てる ので 、 デモ を 見せ て 下さい ",2658
"導入 し たい の です が 、 デモ は あり ます か ？ ",2658
"導入 を 考え て いる の です が 、 デモ を 見せ て いただく こと は でき ます か ？ ",2658
"お客様 用 です か ？ ",2727
~~~~

### 分類器作成

create関数によりAPIコールを実行すると、分類器インスタンスが、Bluemix上に１件生成されます。

（API呼出件数／インスタンス件数は、いずれも課金対象となるので、無償で評価時は実行回数に注意）

In [3]:
test_data_file = 'resources/train_myope_info_conversation.csv'
f = open(test_data_file, 'rt') # これは確認用
print(f.read()[:100])
f.close()

"My - ope ってな ん です か ？ ",2653
"どんな こと が 出来る の ？ ",2654
"どう やっ て 会話 を 覚え させる の ？ ",2655
"どんな 企業 が 使っ 


In [4]:
classifier_id = None
status = None
with open(test_data_file, 'rb') as training_data:
  classifier = natural_language_classifier.create(
    training_data=training_data,
    name='My Classfier',
    language='ja'
  )
  classifier_id = classifier['classifier_id']
  status = classifier['status']

### 分類器作成状況の照会

分類器インスタンスの生成には、時間がかかるため、作成が完了したかどうかを知るには、下記のような状況照会のためのAPIコールを実行する必要がございます。

（API呼出件数は課金対象となるので、無償で評価時は実行回数に注意）

#### 分類器作成が利用できるまで待機

以下の例では、最大３０分間待機します。

今回（土曜日／午前中）は１０分ほどで作成が完了しておりますが、時間帯によって異なるものと考えられます。

In [5]:
from time import sleep
import sys
i = 0
while i < 30:
    statusDict = natural_language_classifier.status(classifier_id)
    status = statusDict['status']
    if status == 'Available':
        break

    print('Classifier [%s] now training, please wait...' % classifier_id)
    sys.stdout.flush()
    sleep(60)
    i += 1

if status == 'Available':
    print('Classifier [%s] training completed.' % classifier_id)
else:
    print('Classifier [%s] training failed.' % classifier_id)

Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] now training, please wait...
Classifier [f5bbbbx174-nlc-4304] training completed.


In [3]:
classifiers = natural_language_classifier.list()
classifiers

{'classifiers': [{'classifier_id': 'f5bbbbx174-nlc-4304',
   'created': '2017-02-18T01:16:40.434Z',
   'language': 'ja',
   'name': 'My Classfier',
   'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f5bbbbx174-nlc-4304'}]}

### 分類実行

classify関数によりAPIコールを実行します。

引数のテキストが分類され、クラスに対するマッチ率が、上位１０クラス分戻ります。

（API呼出件数は課金対象となるので、無償で評価時は実行回数に注意）

In [21]:
question_text = '購入を考えているのですが、デモを見せていただくことは可能でしょうか？'
classesDict = natural_language_classifier.classify(classifier_id, question_text)

APIからは、分類結果がdict形式で戻ります。

In [22]:
classesDict

{'classes': [{'class_name': '2658', 'confidence': 0.9847580249407819},
  {'class_name': '2718', 'confidence': 0.002298623638069411},
  {'class_name': '2723', 'confidence': 0.0012723095005166235},
  {'class_name': '2727', 'confidence': 0.001192302074605971},
  {'class_name': '2657', 'confidence': 0.0011610145369191703},
  {'class_name': '2729', 'confidence': 0.0010245719035502766},
  {'class_name': '2653', 'confidence': 0.000981868634805316},
  {'class_name': '2719', 'confidence': 0.0009525080701032914},
  {'class_name': '2724', 'confidence': 0.0007071948648351854},
  {'class_name': '2662', 'confidence': 0.0006447405270287999}],
 'classifier_id': 'f5bbbbx174-nlc-4304',
 'text': '購入を考えているのですが、デモを見せていただくことは可能でしょうか？',
 'top_class': '2658',
 'url': 'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/f5bbbbx174-nlc-4304'}

ちなみに、クラス2658の回答文は「ありがとうございます！メールフォームからお問い合わせいただければ担当スタッフから連絡させていただきます♪」です。

#### 分類結果の参照

分類されたクラスとマッチ率を表示するコードの例です。

In [23]:
for c in classesDict['classes']:
    print('class name=%s, confidence=%0.1f%%' % (c['class_name'], float(c['confidence'])*100.0))

class name=2658, confidence=98.5%
class name=2718, confidence=0.2%
class name=2723, confidence=0.1%
class name=2727, confidence=0.1%
class name=2657, confidence=0.1%
class name=2729, confidence=0.1%
class name=2653, confidence=0.1%
class name=2719, confidence=0.1%
class name=2724, confidence=0.1%
class name=2662, confidence=0.1%


#### 違った質問で再試行

ついでにレスポンス時間も計測してみました。

In [14]:
from time import time

def predict_class(natural_language_classifier, idx, question_text):
    t0 = time()

    classifiers = natural_language_classifier.list()
    classifier_id = classifiers['classifiers'][idx]['classifier_id']
    
    classesDict = natural_language_classifier.classify(classifier_id, question_text)
    topclass = classesDict['classes'][0]

    print('class name=%s, confidence=%0.1f%%' % (topclass['class_name'], float(topclass['confidence'])*100.0))
    print("done in %0.3fs." % (time() - t0))

    return topclass

以下の例では、「どんな」と「どのような」で、分類されるクラスが異なるという結果になってしまいました。

テストデータを参照した限りでは、正解と思われるのは2663（下）のほうです。

In [15]:
question_text = 'どんなサーバー構成でしょうか？'
top_class = predict_class(natural_language_classifier, 0, question_text)

class name=2654, confidence=20.6%
done in 2.670s.


【ご参考：回答】My-ope officeは社内から来る問い合わせに自動的に回答することが出来ます。難しいフローやロジックなどは不要で、質問・回答セットのデータを投入することで自由にカスタマイズすることが出来ます。

In [16]:
question_text = 'どのようなサーバー構成でしょうか？'
top_class = predict_class(natural_language_classifier, 0, question_text)

class name=2663, confidence=30.4%
done in 2.945s.


【ご参考：回答】インフラは実績も多くセキュリティ評価も高いAmazon Web Services(AWS)のサーバを使っております。高可用性で安定しているので安心です♪

#### 違った質問で再試行 (2)

テストデータにない「コールセンター」という単語を使って質問します。正解ではないのですが、それなりに近い予測結果になっています。

In [17]:
question_text = 'コールセンターの代わりとして使えそうでしょうか？'
top_class = predict_class(natural_language_classifier, 0, question_text)

class name=2723, confidence=52.7%
done in 3.320s.


【ご参考：回答】データ投入の代行については、現在サービス検討中です。詳細はメールフォームよりお問い合わせください。

#### 違った質問で再試行 (3)

テストデータとは全く無関係の質問をします。

In [18]:
question_text = '今日の天気を教えていただけますでしょうか？'
top_class = predict_class(natural_language_classifier, 0, question_text)

class name=2717, confidence=27.0%
done in 3.053s.


【ご参考：回答】こんにちは♪

### 分類器削除

remove関数によりAPIコールを実行すると、分類器インスタンスがBluemix上から削除されます。

（API呼出件数は課金対象となるので、無償で評価時は実行回数に注意）

In [19]:
''' 後日、実行予定です
natural_language_classifier.remove(classifier_id)
print('Classifier [%s] removed.' % classifier_id)
'''

" 後日、実行予定です\nnatural_language_classifier.remove(classifier_id)\nprint('Classifier [%s] removed.' % classifier_id)\n"